In [26]:
import tensorflow as tf
import tensorflow.keras as k
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model

In [27]:
encoding_dim = 32
image_input = Input(shape=(784, ))

In [28]:
# encoded
encoded = Dense(encoding_dim, activation="relu")(image_input)

# decoded 
decoded = Dense(784, activation="sigmoid")(encoded)

# autoencoder
autoencoder = Model(image_input, decoded)

In [29]:
# ENCODER 
encoder = Model(image_input, encoded)

# placeholder for an encoded 32-dimensional input
encoded_input = Input(shape=(encoding_dim, ))

# retrieve last later from encoder
decoder_layer = autoencoder.layers[-1]

# DECODER
decoder = Model(encoded_input, decoder_layer(encoded_input))

In [30]:
autoencoder.compile(optimizer="adadelta", loss="binary_crossentropy")

In [31]:
from tensorflow.keras.datasets import mnist
import numpy as np

(x_train, _), (x_test, _) = mnist.load_data()

In [34]:
x_train = x_train.astype("float32")/255.
x_test = x_test.astype("float32")/255.
x_train = x_train.reshape((len(x_train), np.prod(x_train.shape[1:])))
x_test = x_test.reshape((len(x_test), np.prod(x_test.shape[1:])))
print(x_train.shape, x_test.shape)

(60000, 784) (10000, 784)


In [35]:
autoencoder.fit(
    x_train, x_train,
    epochs=50,
    batch_size=256,
    shuffle=True,
    validation_data=(x_test, x_test)
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 1s 23us/sample - loss: 0.6931 - val_loss: 0.6931
Epoch 2/50
60000/60000 [==============================] - 1s 16us/sample - loss: 0.6930 - val_loss: 0.6930
Epoch 3/50
60000/60000 [==============================] - 1s 16us/sample - loss: 0.6930 - val_loss: 0.6929
Epoch 4/50
60000/60000 [==============================] - 1s 16us/sample - loss: 0.6929 - val_loss: 0.6928
Epoch 5/50
60000/60000 [==============================] - 1s 16us/sample - loss: 0.6928 - val_loss: 0.6928
Epoch 6/50
60000/60000 [==============================] - 1s 15us/sample - loss: 0.6927 - val_loss: 0.6927
Epoch 7/50
60000/60000 [==============================] - 1s 16us/sample - loss: 0.6926 - val_loss: 0.6926
Epoch 8/50
60000/60000 [==============================] - 1s 16us/sample - loss: 0.6926 - val_loss: 0.6925
Epoch 9/50
60000/60000 [==============================] - 1s 16us/sample - loss: 0.6925 - val_

In [36]:
# encode and decode some digits
# note that we take them from the *test* set
encoded_imgs = encoder.predict(x_test)
decoded_imgs = decoder.predict(encoded_imgs)

In [40]:
import matplotlib.pyplot as plt
%matplotlib inline
"""
n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()
"""

'\nn = 10  # how many digits we will display\nplt.figure(figsize=(20, 4))\nfor i in range(n):\n    # display original\n    ax = plt.subplot(2, n, i + 1)\n    plt.imshow(x_test[i].reshape(28, 28))\n    plt.gray()\n    ax.get_xaxis().set_visible(False)\n    ax.get_yaxis().set_visible(False)\n\n    # display reconstruction\n    ax = plt.subplot(2, n, i + 1 + n)\n    plt.imshow(decoded_imgs[i].reshape(28, 28))\n    plt.gray()\n    ax.get_xaxis().set_visible(False)\n    ax.get_yaxis().set_visible(False)\nplt.show()\n'

In [42]:
from tensorflow.keras.layers import (
    Input,
    Dense,
    Conv2D,
    MaxPooling2D,
    UpSampling2D
)
from tensorflow.keras.models import Model

In [44]:
# CONV ENCODER
image_input = Input(shape=(28, 28, 1))
# 28x28x1
x = Conv2D(16, (3, 3), activation="relu", padding="same")(image_input)
x = MaxPooling2D((2, 2), padding="same")(x)
# 14x14x16
x = Conv2D(8, (3, 3), activation="relu", padding="same")(x)
x = MaxPooling2D((2, 2), padding="same")(x)
# 7x7x8
x = Conv2D(8, (3, 3), activation="relu", padding="same")(x)
encoded = MaxPooling2D((2, 2), padding="same")(x)

# 4x4x8 ~ 128 dimensions
# CONV DECODER
x = Conv2D(8, (3, 3), activation="relu", padding="same")(encoded)
x = UpSampling2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation="relu", padding="same")(x)
x = UpSampling2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation="relu")(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation="sigmoid", padding="same")(x)

autoencoder = Model(image_input, decoded)
autoencoder.compile(optimizer="adadelta", loss="binary_crossentropy")

In [46]:
from keras.datasets import mnist
import numpy as np

(x_train, _), (x_test, _) = mnist.load_data()

x_train = x_train.astype('float32') / 255.
x_test = x_test.astype('float32') / 255.
x_train = np.reshape(x_train, (len(x_train), 28, 28, 1))
x_test = np.reshape(x_test, (len(x_test), 28, 28, 1)) 

In [51]:
from tensorflow.keras.callbacks import TensorBoard
autoencoder.fit(
    x_train, x_train,
    epochs=50,
    batch_size=128,
    shuffle=True,
    validation_data=(x_test, x_test),
    callbacks=[TensorBoard(
        log_dir="tmp/autoencoder/", 
        write_graph=True, 
        write_images=True
    )]
)

Train on 60000 samples, validate on 10000 samples
Epoch 1/50
60000/60000 [==============================] - 36s 595us/sample - loss: 0.2488 - val_loss: 0.2479
Epoch 2/50
60000/60000 [==============================] - 36s 599us/sample - loss: 0.2467 - val_loss: 0.2459
Epoch 3/50
60000/60000 [==============================] - 36s 598us/sample - loss: 0.2448 - val_loss: 0.2440
Epoch 4/50
60000/60000 [==============================] - 36s 604us/sample - loss: 0.2430 - val_loss: 0.2423
Epoch 5/50
60000/60000 [==============================] - 35s 591us/sample - loss: 0.2413 - val_loss: 0.2406
Epoch 6/50
60000/60000 [==============================] - 36s 599us/sample - loss: 0.2397 - val_loss: 0.2390
Epoch 7/50
60000/60000 [==============================] - 36s 595us/sample - loss: 0.2382 - val_loss: 0.2375
Epoch 8/50
60000/60000 [==============================] - 36s 599us/sample - loss: 0.2367 - val_loss: 0.2361
Epoch 9/50
60000/60000 [==============================] - 36s 597us/sample - l